# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [33]:
import pandas as pd
import pymongo

In [34]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [35]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Nettoyer les données

In [36]:
US_category = pd.read_json("./data/US_category_id.json")
US_category=US_category.rename(columns = {'kind':'kindUS_category'})
US_category1 = pd.concat([US_category.drop(["items"],axis=1),US_category["items"].apply(pd.Series)], axis=1)
US_category1.head()

,kindUS_category,etag,kind,etag,id,snippet
0,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."
1,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."
2,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."
3,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."
4,youtube#videoCategoryListResponse,"""m2yskBQFythfE4irbTIeOgYYfBU/S730Ilt-Fi-emsQJv...",youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,"{'channelId': 'UCBR8-60-B28hp2BmDPdntcQ', 'tit..."


Les deux bases de données ont été mergées.

### Importer les données

In [37]:
collection.drop()
collection.insert_many(df_youtube.to_dict('records'))

## Question 1  

Nous devons récupérer les vidéos de la marque Apple. Nous utilisons `find()`.

In [38]:
cur = collection.find({"channel_title":"Apple"})
list(cur)

[{'_id': ObjectId('5fce459a00a6bd21a9951f56'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 7860119,
  'likes': 185853,
  'dislikes': 26679,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 13.09},
 {'_id': ObjectId('5fce459a00a6bd21a9952020'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 12200526,
  'likes': 258842,
  'dislikes': 44339,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 14.09},
 {'_id': Obje

## Question 2

Nous cherchons le nombre de catégories différentes. En partant du principe que chaque catégorie à son propre `category_id`, on trouve alors le résultat suivant.

In [39]:
len(database.youtube.distinct('category_id'))

16

## Question 3

Il faudrait utiliser `update` pour mettre à jour chacun des tags. Mais avant, nous devons découper les tags en listes.

In [48]:
df_youtube['tags']=df_youtube['tags'].to_list()

result = collection.update_many({"tags":"|"}, {"$set":{"tags":True}})

## Question 4

Nous récupérons les 10 vidéos les plus vues. Pour ce faire, il a été question de trier par ordre décroissant les vidéos en fonction de leur nombre de vues.

In [49]:
cur = collection.find().sort([("views", -1)]).limit(10)
list(cur)

[{'_id': ObjectId('5fce459a00a6bd21a99526c8'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09},
 {'_id': ObjectId('5fce459a00a6bd21a99525f3'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 38013692,
  'likes': 1950475,
  'dislikes': 75294,
  'comment_total': 713293,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 21.09},
 {'_id': ObjectId('5fce459a00a6bd21a9952006'),
  'video_id': 'tt2k8PGm-TI',
  'title': 'ZAYN - Dusk Till Dawn ft. Sia',
  'channel_title': 'ZaynVEVO',
  'category_id': 10,
  'tags': 'Dusk Till Dawn|Pop|RCA Records 

## Question 5

Nous voulons compter le nombre moyen de vues en fonction de la catégorie. Pour ce faire, on utilise en effet une fonction d'aggrégation.

In [51]:
cur = collection.aggregate([{"$group" : {"_id" : "$category_id", "nombre dans categorie" : {"$avg" : "$views"}}}])
list(cur)

[{'_id': 17, 'nombre dans categorie': 728434.3243902439},
 {'_id': 26, 'nombre dans categorie': 537665.1806674339},
 {'_id': 29, 'nombre dans categorie': 1110334.2142857143},
 {'_id': 10, 'nombre dans categorie': 1176553.612},
 {'_id': 25, 'nombre dans categorie': 540955.5569823436},
 {'_id': 1, 'nombre dans categorie': 1039472.6560846561},
 {'_id': 27, 'nombre dans categorie': 547582.4700598803},
 {'_id': 22, 'nombre dans categorie': 971532.7573696146},
 {'_id': 20, 'nombre dans categorie': 681081.8170731707},
 {'_id': 43, 'nombre dans categorie': 8492.5},
 {'_id': 2, 'nombre dans categorie': 607693.8706896552},
 {'_id': 28, 'nombre dans categorie': 924730.548828125},
 {'_id': 23, 'nombre dans categorie': 1240073.294039735},
 {'_id': 19, 'nombre dans categorie': 464041.0833333333},
 {'_id': 24, 'nombre dans categorie': 1154868.4397251718},
 {'_id': 15, 'nombre dans categorie': 651404.3879310344}]

## Question 6 

Il est ici question de récupérer les 10 chaines Youtube avec la plus grande moyenne de likes. Pour ce faire, on a trié par ordre décroissant en fonction du nombre de likes.

On réalise ensuite une aggrégation.

In [52]:
cur = collection.find().sort([("likes", -1)]).limit(10)
cur2 = collection.aggregate([{"$group" : {"_id" : "$channel_title", "nombre moyen de likes" : {"$avg" : "$likes"}}}])
list(cur)

[{'_id': ObjectId('5fce459a00a6bd21a99526c8'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09},
 {'_id': ObjectId('5fce459a00a6bd21a99525f3'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 38013692,
  'likes': 1950475,
  'dislikes': 75294,
  'comment_total': 713293,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 21.09},
 {'_id': ObjectId('5fce459a00a6bd21a99524f5'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄'